<h1>Limpeza e Preparação dos Dados - Ch.07
    

Sabe-se que as tarefas de carga, limpeza, transformação e reorganização dos dados ocupam em média 80% do tempo do analista. Muitos pesquisadores preferem fazer o processamento ad hoc de um formato de dados para outro com uma lingaugem de programação de uso geral, como o Python.

<h2>Tratando Dados Ausentes

O valor de ponto flutuante NaN é conhecido como *valor sentinela* e pode ser facilmente detectado


In [54]:
import pandas as pd
import numpy as np

string_data = pd.Series(["Palavra_01","Palavra_02", np.nan, "Abacate"])
print(string_data)
print(string_data.isnull())

0    Palavra_01
1    Palavra_02
2           NaN
3       Abacate
dtype: object
0    False
1    False
2     True
3    False
dtype: bool


Dados ausentes ou existentes e não observados são conhecidos como NA (*Not Available*). O valor embutido *None* de Python é tratado como NA em arrays de objetos.

In [2]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

Outras funções que podem ser úteis são a **dropna() e fillna()**

In [3]:
from numpy import nan as NA

#dropna

data = pd.Series([1,NA,3.5,NA,7])
print(data)
print(data.dropna())

#isto é equivalente à:
cleaned = data[data.notnull()]
print(cleaned)

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64
0    1.0
2    3.5
4    7.0
dtype: float64
0    1.0
2    3.5
4    7.0
dtype: float64


No caso de Dataframes, por padrão, exclui-se as linhas com qualquer valor NA. Para excluir linhas que contenham somente NAs, precisamos especificar o parâmetro *how*

In [4]:
data = pd.DataFrame([[1.,5.,3.],[1,NA,NA],[NA,NA,NA],[NA,6.5,3.]])
print(data)
print()

#Limpando linhas padrão
cleaned = data.dropna()
print(cleaned)
print()

#Limpando linhas só NA
print(data.dropna(how='all'))
print()

#Limpando colunas só com NA
data[3] = NA
print(data)
print(data.dropna(how='all',axis=1))

     0    1    2
0  1.0  5.0  3.0
1  1.0  NaN  NaN
2  NaN  NaN  NaN
3  NaN  6.5  3.0

     0    1    2
0  1.0  5.0  3.0

     0    1    2
0  1.0  5.0  3.0
1  1.0  NaN  NaN
3  NaN  6.5  3.0

     0    1    2   3
0  1.0  5.0  3.0 NaN
1  1.0  NaN  NaN NaN
2  NaN  NaN  NaN NaN
3  NaN  6.5  3.0 NaN
     0    1    2
0  1.0  5.0  3.0
1  1.0  NaN  NaN
2  NaN  NaN  NaN
3  NaN  6.5  3.0


Supondo que queremos manter as linhas que contenham determinado números de observações, podemos especificar o argumento *thresh*

In [5]:
df = pd.DataFrame(np.random.randn(7,3))
df.iloc[:4,0] = NA
df.iloc[:2,2] = NA
print(df)
print()

print(df.dropna(thresh=2))
print(df.dropna(thresh=2,axis=1))

          0         1         2
0       NaN -0.096414       NaN
1       NaN  1.784732       NaN
2       NaN  0.134634  0.358632
3       NaN -0.458480  0.575575
4  0.213576  1.214341 -0.766193
5 -0.642570 -1.191114  0.127508
6  0.555415 -0.200536 -2.562787

          0         1         2
2       NaN  0.134634  0.358632
3       NaN -0.458480  0.575575
4  0.213576  1.214341 -0.766193
5 -0.642570 -1.191114  0.127508
6  0.555415 -0.200536 -2.562787
          0         1         2
0       NaN -0.096414       NaN
1       NaN  1.784732       NaN
2       NaN  0.134634  0.358632
3       NaN -0.458480  0.575575
4  0.213576  1.214341 -0.766193
5 -0.642570 -1.191114  0.127508
6  0.555415 -0.200536 -2.562787


Há diferentes formas para se usar a função **fillna()** . Passando um valor único, um dicionário de valores, ou fazendo forward fill e backward fiil.

In [6]:
df.fillna(0)

,0,1,2
0,0.000000,-0.096414,0.000000
1,0.000000,1.784732,0.000000
2,0.000000,0.134634,0.358632
3,0.000000,-0.458480,0.575575
4,0.213576,1.214341,-0.766193
5,-0.642570,-1.191114,0.127508
6,0.555415,-0.200536,-2.562787


In [7]:
df.fillna({0:0, 2:True})

,0,1,2
0,0.000000,-0.096414,True
1,0.000000,1.784732,True
2,0.000000,0.134634,0.358632
3,0.000000,-0.458480,0.575575
4,0.213576,1.214341,-0.766193
5,-0.642570,-1.191114,0.127508
6,0.555415,-0.200536,-2.56279


In [8]:
df.iloc[0,0] = True
df.iloc[0,2] = 'Não encontrado'
#com implace, não criamos uma cópia, mas substituimos no local
_ = df.fillna(method = 'ffill', inplace = True)
df

,0,1,2
0,True,-0.096414,Não encontrado
1,True,1.784732,Não encontrado
2,True,0.134634,0.358632
3,True,-0.458480,0.575575
4,0.213576,1.214341,-0.766193
5,-0.64257,-1.191114,0.127508
6,0.555415,-0.200536,-2.56279


In [9]:
df.fillna(method='bfill', limit = 2)

,0,1,2
0,True,-0.096414,Não encontrado
1,True,1.784732,Não encontrado
2,True,0.134634,0.358632
3,True,-0.458480,0.575575
4,0.213576,1.214341,-0.766193
5,-0.64257,-1.191114,0.127508
6,0.555415,-0.200536,-2.56279


<h2> Transformação de Dados

<h3> Removendo Duplicatas

In [10]:
data = pd.DataFrame({'k1': ['azul','amarelo'] * 2 + ['amarelo'],
                 'k2': ['preto']*2 + ['branco'] * 3})

#Removendo linhas duplicadas:
print("Antes:\n", data)
print("\nDepois:\n", data.drop_duplicates())

#Removendo valores duplicados de uma coluna específica:
print("\nDepois_02\n", data.drop_duplicates(['k1']))
print("\nDepois_03\n", data.drop_duplicates(['k2']))

#Por padrão, drop_duplicates() preserva o primeiro valor, poderíamos manter o último também:
print('\nDepois_04\n', data.drop_duplicates(['k1'],keep='last'))

Antes:
         k1      k2
0     azul   preto
1  amarelo   preto
2     azul  branco
3  amarelo  branco
4  amarelo  branco

Depois:
         k1      k2
0     azul   preto
1  amarelo   preto
2     azul  branco
3  amarelo  branco

Depois_02
         k1     k2
0     azul  preto
1  amarelo  preto

Depois_03
      k1      k2
0  azul   preto
2  azul  branco

Depois_04
         k1      k2
2     azul  branco
4  amarelo  branco


<h3>Transformando Dados usando uma Função de Mapeamento

In [12]:
data = pd.DataFrame({'food': ['bacon','pulled_pork','bacon','pastrami', 'corned beef', 'bacon', 'honey ham', 'nova lox'],
                   'weight': [4.0, 3.0, 12.0, 10.0, 5.0, 4.7, 20.0, 12.0]})

#dicionário de mapeamento para adição de nova coluna
meat_to_animal = {'bacon': 'pig',
                 'pulled_pork': 'pig',
                 'pastrami': 'cow',
                 'corned beef': 'cow',
                 'honey ham': 'pig',
                 'nova lox': 'salmon'}

#mapeando usando o dicionário
data['animal'] = data['food'].map(meat_to_animal)
data

,food,weight,animal
0,bacon,4.0,pig
1,pulled_pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,10.0,cow
4,corned beef,5.0,cow
5,bacon,4.7,pig
6,honey ham,20.0,pig
7,nova lox,12.0,salmon


<h3> Substituindo Valores

Ao utilizarmos números como *valores sentinela*, podemos usar replace() para substituir esses outliers:

In [13]:
data = pd.Series([1,-999,2,-999,3,-1000])
print("Antes\n", data)
print("Depois\n", data.replace([-999,-1000],np.nan))

Antes
 0       1
1    -999
2       2
3    -999
4       3
5   -1000
dtype: int64
Depois
 0    1.0
1    NaN
2    2.0
3    NaN
4    3.0
5    NaN
dtype: float64


<h3>Transformando e Renomeando os Índices

In [14]:
#Transformando

data = pd.DataFrame(np.arange(12).reshape(3,4),
                   index = ['Ohio','Colorado','New York'],
                   columns = ['one','two','three','four'])
print(data)

transform = lambda x:(x.upper()+"'s")
data.index = data.index.map(transform)
print(data)

          one  two  three  four
Ohio        0    1      2     3
Colorado    4    5      6     7
New York    8    9     10    11
            one  two  three  four
OHIO's        0    1      2     3
COLORADO's    4    5      6     7
NEW YORK's    8    9     10    11


In [15]:
#Renomeando sem transformar
data.rename(index = str.title, columns=str.upper)

#Renomeando os eixos e fornevendo novos valores a partir de um dicionário
data.rename(index={"OHIO's":'Rio de Janeiro'},
           columns = {'three': 'Infinity'},
           inplace = True)
data

,one,two,Infinity,four
Rio de Janeiro,0,1,2,3
COLORADO's,4,5,6,7
NEW YORK's,8,9,10,11


<h3>Discretização e Compartimentalização (binning)

Dados contínuos são frequentemente *discretizados* ou colocados em compartimentos (bins), isto é, faixa de valores.

In [16]:
ages = [20,78,22,25,27,28,32,61,45,54,33,24,29,31,72]

#compartimentalizando as idades
bins = [18,25,35,60,100]
cats = pd.cut(ages,bins)
print()
print(cats)
print()
print(cats.codes)
print()
print(cats.categories)
print()
pd.value_counts(cats)


[(18, 25], (60, 100], (18, 25], (18, 25], (25, 35], ..., (25, 35], (18, 25], (25, 35], (25, 35], (60, 100]]
Length: 15
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

[0 3 0 0 1 1 1 3 2 2 1 0 1 1 3]

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')



(25, 35]     6
(18, 25]     4
(60, 100]    3
(35, 60]     2
dtype: int64

In [17]:
#renomeando as faixas etárias
group_names = ['Youth','YoungAdult','MiddleAged','Senior']
pd.cut(ages,bins,labels=group_names)

['Youth', 'Senior', 'Youth', 'Youth', 'YoungAdult', ..., 'YoungAdult', 'Youth', 'YoungAdult', 'YoungAdult', 'Senior']
Length: 15
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

In [18]:
#se ao invés de passarmos uma lista de bins, passarmos um único valor. Ele divide por esse valor o range.
data = np.random.randn(20)
pd.cut(data,4,precision=2)

[(-0.97, 0.2], (-2.15, -0.97], (-0.97, 0.2], (0.2, 1.38], (-2.15, -0.97], ..., (0.2, 1.38], (-2.15, -0.97], (0.2, 1.38], (0.2, 1.38], (1.38, 2.55]]
Length: 20
Categories (4, interval[float64]): [(-2.15, -0.97] < (-0.97, 0.2] < (0.2, 1.38] < (1.38, 2.55]]

In [19]:
#para dividir de acordo com o número de quantis da amostra, ou seja, ter quantidade de pontos mais ou menos homogêneas por intervalo, usa-se qcut()
pd.qcut(data,4)

[(-0.63, 0.0113], (-2.147, -0.63], (-2.147, -0.63], (0.763, 2.555], (-2.147, -0.63], ..., (0.763, 2.555], (-2.147, -0.63], (0.0113, 0.763], (0.0113, 0.763], (0.763, 2.555]]
Length: 20
Categories (4, interval[float64]): [(-2.147, -0.63] < (-0.63, 0.0113] < (0.0113, 0.763] < (0.763, 2.555]]

<h3>Detectando e filtrando valores discrepantes (outliers)

In [21]:
data = pd.DataFrame(np.random.randn(1000,4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.022306,0.010216,-0.033156,-0.020897
std,1.006307,1.035148,1.029990,1.030165
min,-3.094382,-2.888530,-3.816744,-3.460110
25%,-0.688648,-0.714461,-0.696340,-0.740291
50%,-0.015825,0.006637,-0.024592,-0.069496
75%,0.696998,0.694654,0.613043,0.716179
max,3.190657,3.182304,3.143341,3.459791


Vamos supor que queiramos cortar todos os números que excederem 3 o valor absoluto em uma das colunas. Poderíamos usar uma *boolean mask* diretamente

In [23]:
col = data[2]
col[np.abs(col) > 3]

154    3.143341
348    3.111538
564   -3.816744
Name: 2, dtype: float64

Para qualquer coluna *any(1)*:

In [29]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
154,1.940561,1.834697,3.143341,-0.021417
163,0.363725,-0.443479,-1.704588,3.459791
254,-3.094382,-0.460754,1.109118,-1.488145
269,1.721978,3.166721,0.108274,-0.035787
341,0.317763,3.018608,1.637385,-0.595749
348,-0.503207,0.180877,3.111538,1.365694
564,1.267288,0.185142,-3.816744,0.368975
575,0.257193,-0.548856,1.019397,3.137774
610,-0.846591,3.182304,-1.759690,0.704279
648,-2.514154,1.020331,0.695502,3.236597


Se quiséssemos limitar qualquer valor fora do módulo de 3 a 3, poderíamos usar a função *sign()* nos outliers. 

ELa retorna -1 para valores negativos e 1 para valores positivos. 

Em seguida, multiplicamos o resultado final por três

In [31]:
data[(np.abs(data) > 3)] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.022210,0.009849,-0.032594,-0.021272
std,1.005435,1.034054,1.026538,1.026134
min,-3.000000,-2.888530,-3.000000,-3.000000
25%,-0.688648,-0.714461,-0.696340,-0.740291
50%,-0.015825,0.006637,-0.024592,-0.069496
75%,0.696998,0.694654,0.613043,0.716179
max,3.000000,3.000000,3.000000,3.000000


<h3>Permutação e Amostragem Aleatória

A permutação é uma reordenação aleatória de valores. Podemos criar um array com os índices de permutação chamando a fundação *oermutation()* da numpy.

In [37]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5,4)))
df.head()

sampler = np.random.permutation(5)
sampler

,0,1,2,3
2,8,9,10,11
4,16,17,18,19
3,12,13,14,15
1,4,5,6,7
0,0,1,2,3


Para aplicar o array de permutação no DF, basta usar o método *take()*

In [38]:
df.take(sampler)

,0,1,2,3
2,8,9,10,11
4,16,17,18,19
3,12,13,14,15
1,4,5,6,7
0,0,1,2,3


Para conseguir uma amostragem aleatória das linhas de um DF é só usar o método embutido *sample()*

In [44]:
df.sample(n=3) #seleciona 3 linhas

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


In [58]:
df.sample(frac=0.50) #seleciona 50% do DF

,0,1,2,3
3,12,13,14,15
2,8,9,10,11


<h3>Calculando Variáveis Indicadoras (dummy)

Variáveis dummy são variáveis binárias criadas para representar uma variável com duas ou mais categorias.
Por exemplo, se quiséssemos incluir a variável sexo em um modelo de regressão teríamos que transformar artificialmente a variável sexo em uma variável dummy, deste modelo teríamos:

dummy_sexo = 1 em caso de sexo feminino

dummy_sexo = 0 em caso de sexo masculino

O pandas já tem uma função embutida para criar dummies a partir de um DF. Vamos ver?

In [62]:
df = pd.DataFrame({'key' : ['b','b','a','c','a','a','b','c'],
                  'data': range(8)}
                 )
pd.get_dummies(df['key'],prefix='key')

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,1,0,0
6,0,1,0
7,0,0,1


Se uma linha do DF pertencer a várias categorias, a situação se torna um pouco mais complicada. 

Vamos ver um exemplo do Movie Lens que reflete esse caso no gênero dos filmes.

In [63]:
mnames = ['movie_id','title','genres']

movies = pd.read_table('examples/movies.dat',sep='::',header=None,names=mnames)
movies[:10]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\parsers.py:765: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return read_csv(**locals())


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [64]:
all_genres = []
[all_genres.extend(x.split('|')) for x in movies.genres] 
#obs. como pode haver mais de um gênero por filme, temos que usar extend ao invés de append(append, seria para um único elemento)
genres = pd.unique(all_genres)
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

Uma maneira de começar um dataframe indicador é começar com um DataFrame contendo somente zeros:

In [80]:
#a dimensionalidade depende do número de títulos e de gêneros
zero_matrix = np.zeros((len(movies), len(genres)))

#Aqui construímos o 'esqueleto'
dummies = pd.DataFrame(zero_matrix, columns = genres)

#Aqui temos a função geradora com o gênero de cada filme (para visualizá-la, só converter em dicionário)
movies_gen = enumerate(movies.genres)

#Agora iteramos por cada linha para definir os dummies
for i_title, j_genre in movies_gen:
    indices = dummies.columns.get_indexer(j_genre.split('|'))
    dummies.iloc[i_title,indices] = 1

movies_classified = movies.join(dummies.add_prefix('Genre_'))
movies_classified[:2]

,movie_id,title,genres,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Adventure,Genre_Fantasy,Genre_Romance,Genre_Drama,...,Genre_Crime,Genre_Thriller,Genre_Horror,Genre_Sci-Fi,Genre_Documentary,Genre_War,Genre_Musical,Genre_Mystery,Genre_Film-Noir,Genre_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
#Podemos combinar dummies com bins, conforme o exemplo abaixo, para montrar uma matriz de categorização
np.random.seed(12345)

values = np.random.rand(10)
values

bins = [0,0.2,0.4,0.6,0.8,1]

pd.get_dummies(pd.cut(values,bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


<h2>Manipulação de Strings

<h3>Métodos de Objetos String

In [5]:
val = 'a,b, qualquer coisa'
val.split(',')

['a', 'b', ' qualquer coisa']

In [8]:
pieces = [s.strip() for s in val.split(',')]
pieces

['a', 'b', 'qualquer coisa']

In [9]:
primeiro, segundo, terceiro = pieces
primeiro + '::' + segundo + '::' + terceiro

'a::b::qualquer coisa'

In [10]:
'::'.join(pieces)

'a::b::qualquer coisa'

In [13]:
val.count(',')

2

In [14]:
val.replace(',','::')

'a::b:: qualquer coisa'

<h3>Expressões Regulares

In [15]:
import re
text = 'fazendo    teste\t de   \tseparação'
re.split('\s+', text)

['fazendo', 'teste', 'de', 'separação']

In [17]:
#para criar um objeto RE reutilizável, poderíamos compilá-lo manualmente
regex = re.compile('\s+')
regex.split(text)

['fazendo', 'teste', 'de', 'separação']

**Importante**: criar um objeto regex é altamente recomendável caso formos reutilizá-lo várias vezes. Isso economiza ciclos de CPU.

In [18]:
#para ver os padrões usados pela regex para separar as strings, podemos usar findall()
regex.findall(text)

['    ', '\t ', '   \t']

In [43]:
text = """David david@gmail.com
Pedro pedro@hotmail.com
Julia julia@ycamara.gov
Mario mario@ufscar.br
"""

pattern = r'[A-Z]+@[A-Z]+\.[A-Z]{2,4}'
regex = re.compile(pattern, flags = re.IGNORECASE)
regex.findall(text)

['david@gmail.com',
 'pedro@hotmail.com',
 'julia@ycamara.gov',
 'mario@ufscar.br']

In [28]:
print(regex.sub('E-MAIL CONFIDENCIAL',text))

David E-MAIL CONFIDENCIAL
Pedro E-MAIL CONFIDENCIAL
Julia E-MAIL CONFIDENCIAL
Mario E-MAIL CONFIDENCIAL



Para gerar um padrão que segmente as partes, é só colocar parênteses nos grupos que queremos extrair individualmente

In [44]:
pattern2 = r'([A-Z]+)@([A-Z]+)\.([A-Z]{2,4})'
regex2 = re.compile(pattern2, flags = re.IGNORECASE)
regex2.findall(text)
print(regex2.sub(r'Username: \1, Domínio: \2, Sufixo: \3',text))

David Username: david, Domínio: gmail, Sufixo: com
Pedro Username: pedro, Domínio: hotmail, Sufixo: com
Julia Username: julia, Domínio: ycamara, Sufixo: gov
Mario Username: mario, Domínio: ufscar, Sufixo: br



<h3>Strings Vetorizadas no Pandas

Vamos manipular os dados anteriores para encaixá-los em um objeto tipo *Series*:

In [56]:
nomes = ('David','Pedro','Julia','Mario')
data = dict(zip(nomes,regex.findall(text)))

data = pd.Series(data)
data

David      david@gmail.com
Pedro    pedro@hotmail.com
Julia    julia@ycamara.gov
Mario      mario@ufscar.br
dtype: object

Objetos Series tem métodos orientados a arrays para operações em string ignorando valores NA. Eles são acessados por meio do atributo .str

In [57]:
data.str.contains('gmail')

David     True
Pedro    False
Julia    False
Mario    False
dtype: bool

In [59]:
print(pattern)
data.str.findall(pattern,flags=re.IGNORECASE)

[A-Z]+@[A-Z]+\.[A-Z]{2,4}


David      [david@gmail.com]
Pedro    [pedro@hotmail.com]
Julia    [julia@ycamara.gov]
Mario      [mario@ufscar.br]
dtype: object